In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE68526"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE68526"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE68526.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE68526.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE68526.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral blood transcriptome profiles from an RNA Pilot Study within the United States Health and Retirement Study (HRS)"
!Series_summary	"Individual differences in peripheral blood transcriptomes in older adults as a function of demographic, socio-economic, psychological, and health history characteristics."
!Series_overall_design	"Gene expression profiling was carried out on peripheral blood RNA samples collected from 121 community dwelling older adults participating in the United States Health and Retirement Study.  In addition to basic demographic characteristics (age, sex, race/ethnicity), participants were also assessed on health-related characteristics (body mass index/BMI; history of smoking or heavy alcohol consumption; history of chronic illnesses such as diabetes, cardiovascular disease, cancer, stroke), household annual income (log transformed), and measures of loneliness (UCLA Loneliness Scale; Russell D, Peplau LA, Cutrona CE: The 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any
import json

# 1. Gene Expression Data Availability
# Based on the background information, this is a peripheral blood transcriptome study
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Examine the sample characteristics for trait, age, and gender

# Stroke data - need to examine field 5 which contains 'diabcvdcastr'
# This appears to indicate diabetes, cardiovascular disease, cancer, and stroke
trait_row = 5

# Age is available in row 0
age_row = 0

# Gender is available in row 1 (coded as female: 0/1)
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values - diabcvdcastr field contains stroke information"""
    if value is None or pd.isna(value) or 'missing' in str(value).lower():
        return None
    
    # The field 'diabcvdcastr' represents multiple conditions including stroke
    # Based on the name 'diabcvdcastr', the 'str' part likely refers to stroke
    # We're looking at a binary field (1 = has at least one of these conditions, 0 = none)
    try:
        # Extract the value after the colon
        if ':' in str(value):
            val = str(value).split(':', 1)[1].strip()
            # Convert to binary (1 = yes, 0 = no)
            return int(val)
        else:
            return int(value)
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if value is None or pd.isna(value) or 'missing' in str(value).lower():
        return None
    
    try:
        # Extract the numeric part after the colon
        if ':' in str(value):
            val = str(value).split(':', 1)[1].strip()
            return float(val)
        else:
            return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary (0 = female, 1 = male)"""
    if value is None or pd.isna(value) or 'missing' in str(value).lower():
        return None
    
    try:
        # The data is coded as 'female: 0' or 'female: 1'
        # female: 1 means female (so should be 0 in our coding)
        # female: 0 means male (so should be 1 in our coding)
        if ':' in str(value):
            val = str(value).split(':', 1)[1].strip()
            female_indicator = int(val)
            # Convert: female indicator of 1 means female (0), 0 means male (1)
            return 1 - female_indicator
        else:
            # If value is just 0 or 1 without the 'female:' prefix
            female_indicator = int(value)
            return 1 - female_indicator
    except (ValueError, TypeError):
        return None

# 3. Save Metadata for initial filtering
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None (clinical data is available)
if trait_row is not None:
    # We need to reconstruct the clinical_data from the sample characteristics information
    # that was provided in the previous step
    
    # First, let's load the clinical data which should be available from previous steps
    try:
        # Assuming clinical_data is already loaded in this environment
        clinical_data = get_clinical_data()
        
        # Use the function to extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("Clinical data processing step skipped.")


Error processing clinical data: name 'get_clinical_data' is not defined
Clinical data processing step skipped.


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [4]:
# Define the helper function missing from the provided code
def get_feature_data(clinical_df, row_id, feature_name, convert_func):
    """Extract and process a specific feature from clinical data."""
    feature_data = clinical_df.iloc[row_id:row_id+1, :].copy()
    feature_data = feature_data.T
    feature_data.columns = [feature_name]
    feature_data = feature_data.applymap(convert_func)
    return feature_data

# First, let's try to load the clinical data and properly parse GEO data format
import os
import json
import pandas as pd
import numpy as np
import re
from typing import Optional, Callable, Dict, Any

is_gene_available = True  # Assume gene expression data is available by default
trait_row = None
age_row = None
gender_row = None
clinical_data = None

# Check for series_matrix.txt file which is the standard format in GEO datasets
series_matrix_file = os.path.join(in_cohort_dir, "series_matrix.txt")
if os.path.exists(series_matrix_file):
    print(f"Series matrix file found at {series_matrix_file}")
    
    # Parse the series matrix file to extract clinical information
    sample_chars = {}
    sample_titles = []
    
    with open(series_matrix_file, 'r') as f:
        lines = f.readlines()
        
        # Extract sample titles
        for line in lines:
            if line.startswith("!Sample_title"):
                titles = line.strip().split('\t')[1:]
                sample_titles = [t.strip('"') for t in titles]
                
        # Extract and organize sample characteristics by row
        char_lines = [line for line in lines if line.startswith("!Sample_characteristics")]
        
        for i, line in enumerate(char_lines):
            values = line.strip().split('\t')[1:]
            values = [v.strip('"') for v in values]
            sample_chars[i] = values
            
            # Print the first few values to help identify relevant rows
            print(f"Row {i} sample characteristics: {values[:5]}...")
    
    # Create a DataFrame from the sample characteristics
    if sample_chars and sample_titles:
        sample_chars_df = pd.DataFrame(sample_chars, index=sample_titles).T
        clinical_data = sample_chars_df
        
        # Analyze the rows to identify trait, age, and gender
        for i in range(len(clinical_data)):
            unique_values = clinical_data.iloc[i].unique()
            print(f"Row {i} unique values: {unique_values[:5]}")
            
            # Look for trait information
            row_str = ' '.join([str(x).lower() for x in unique_values if str(x) != 'nan'])
            if re.search(r'stroke|disease|control|normal|case|patient|diagnosis', row_str):
                trait_row = i
                print(f"Identified trait row: {i}")
            
            # Look for age information
            if re.search(r'age|years|old', row_str):
                age_row = i
                print(f"Identified age row: {i}")
            
            # Look for gender information
            if re.search(r'gender|sex|male|female', row_str):
                gender_row = i
                print(f"Identified gender row: {i}")
else:
    print(f"Series matrix file not found at {series_matrix_file}")
    
    # Check for clinical_data.csv as a fallback
    clinical_file = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_file):
        clinical_data = pd.read_csv(clinical_file)
        print(f"Clinical data loaded from {clinical_file}")
        
        # Similar analysis would be needed here to identify relevant rows
        # But without seeing the actual data structure, this is challenging

# Define conversion functions with proper handling of formats found in GEO data
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for case)."""
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    if 'stroke' in value_str or 'case' in value_str or 'patient' in value_str or 'disease' in value_str:
        return 1
    elif 'control' in value_str or 'healthy' in value_str or 'normal' in value_str:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous number."""
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    age_match = re.search(r'(\d+(\.\d+)?)', value_str)
    if age_match:
        try:
            return float(age_match.group(1))
        except:
            return None
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    if 'female' in value_str or 'f' == value_str.strip() or 'woman' in value_str:
        return 0
    elif 'male' in value_str or 'm' == value_str.strip() or 'man' in value_str:
        return 1
    else:
        return None

# Determine trait availability
is_trait_available = trait_row is not None

# Use the library function to validate and save cohort info
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Only extract clinical features if trait data is available and clinical data exists
if trait_row is not None and clinical_data is not None:
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical feature extraction skipped - trait data not available or clinical data not loaded.")


Series matrix file not found at ../../input/GEO/Stroke/GSE68526/series_matrix.txt
Clinical feature extraction skipped - trait data not available or clinical data not loaded.


### Step 4: Gene Data Extraction

In [5]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE68526/GSE68526_series_matrix.txt.gz


Gene data shape: (34581, 121)
First 20 gene/probe identifiers:
Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 5: Gene Identifier Review

In [6]:
# Examine the first 20 gene identifiers
# These identifiers (like 'A1BG', 'A2M', 'AAAS') are standard human gene symbols,
# not probe IDs which would typically be numeric or have specific formats (like Affymetrix IDs)
# No mapping is needed as these are already proper human gene symbols

requires_gene_mapping = False


### Step 6: Data Normalization and Linking

In [7]:
# 1. Normalize gene symbols in the obtained gene expression data
# Use the gene_data variable that was already loaded in Step 4
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. No clinical data available (trait_row is None from previous steps)
# We can't proceed with linking clinical and genetic data
print("No trait data available. Can't proceed with linking clinical and genetic data.")

# Create a minimal dataframe for validation purposes 
# (since we need a valid DataFrame when is_final=True)
empty_df = pd.DataFrame({"dummy": [0]})

# Since trait data is not available, the dataset is not usable for our purposes
# We pass is_biased=True to indicate unusable data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=True,  # Setting is_biased to True since missing trait data makes dataset unusable
    df=empty_df,
    note="No stroke-specific trait data available in this dataset. Contains gene expression data from peripheral blood of older adults, but without clear stroke indicators."
)
print("Dataset deemed not usable due to missing trait data.")

Original gene data shape: (34581, 121)
Gene data shape after normalization: (20741, 121)


Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE68526.csv
No trait data available. Can't proceed with linking clinical and genetic data.
Abnormality detected in the cohort: GSE68526. Preprocessing failed.
Dataset deemed not usable due to missing trait data.
